In [1]:
import cv2
import numpy as np
from keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.neighbors import KNeighborsClassifier
import os

# Load ResNet50 model without the top classification layer for feature extraction
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Prepare the KNN classifier
knn = KNeighborsClassifier(n_neighbors=1)

# Paths
dataset_path = "D:\Python\ML\Face recognition system\dataset"
labels = []
encodings = []

# Prepare the dataset
def preprocess_images():
    global encodings, labels
    
    for person_name in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person_name)
        for image_file in os.listdir(person_path):
            img_path = os.path.join(person_path, image_file)
            image = cv2.imread(img_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            resized_image = cv2.resize(image_rgb, (224, 224))  # ResNet50 input size
            image_array = np.expand_dims(resized_image, axis=0)
            image_array = preprocess_input(image_array)

            # Extract features using ResNet50
            features = model.predict(image_array)

            # Store features and labels
            encodings.append(features.flatten())  # Flatten features for KNN
            labels.append(person_name)

# Train the KNN model
def train_knn():
    knn.fit(encodings, labels)
    print("KNN model trained with dataset.")

# Initialize face detector (Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Real-time face recognition function
def recognize_face_in_video():
    cap = cv2.VideoCapture(0)  # Access the webcam
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            # Crop the detected face
            face = frame[y:y+h, x:x+w]
            face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            resized_face = cv2.resize(face_rgb, (224, 224))  # Resize for ResNet50
            face_array = np.expand_dims(resized_face, axis=0)
            face_array = preprocess_input(face_array)

            # Extract features
            face_features = model.predict(face_array)
            face_encoding = face_features.flatten()

            # Predict the person using KNN
            predicted_label = knn.predict([face_encoding])[0]

            # Draw a rectangle around the face and label it
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, predicted_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        # Display the video feed with recognized faces
        cv2.imshow('Face Recognition', frame)

        # Break the loop on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Main execution
preprocess_images()  # Prepare dataset and preprocess images
train_knn()          # Train KNN classifier
recognize_face_in_video()  # Start real-time recognition


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 